In [ ]:
import pandas as pd
import os
import sys
import numpy as np

In [ ]:
folium_path = "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages"
if folium_path not in sys.path:
    sys.path.append(folium_path)

In [ ]:
import folium
from folium.plugins import MeasureControl

# Helper functions

In [ ]:
def plot_map_with_hollow_radii(df, country, loc_type):
    # Filter dataframe by country and type
    filtered_df = df[(df['country_name'] == country) & (df['type'] == loc_type)]
    
    if filtered_df.empty:
        print(f"No data found for country '{country}' and type '{loc_type}'")
        return None
    
    # Get the mean latitude and longitude for the initial map location
    center_lat = filtered_df['latitude_deg'].mean()
    center_lon = filtered_df['longitude_deg'].mean()
    
    # Create a folium map centered at the mean location
    folium_map = folium.Map(location=[center_lat, center_lon], zoom_start=6)
    
    # Add markers and hollow circles for each point in the filtered dataframe
    for _, row in filtered_df.iterrows():
        # Add marker
        folium.Marker(
            location=[row['latitude_deg'], row['longitude_deg']],
            popup=f"Location: {row['latitude_deg']}, {row['longitude_deg']}"
        ).add_to(folium_map)
        
        # Add red circle with 200 km radius, no fill
        folium.Circle(
            radius= 200 * 1000,  # 200 km in meters
            location=[row['latitude_deg'], row['longitude_deg']],
            color='red',
            fill=False,  # No fill for visibility
            weight=1
        ).add_to(folium_map)
        """
        # Add blue circle with 800 km radius, no fill
        folium.Circle(
            radius=800, #* 1000,  # 800 km in meters
            location=[row['latitude_deg'], row['longitude_deg']],
            color='blue',
            fill=False  # No fill for visibility
        ).add_to(folium_map)
        """
    # Add a measuring tool for interactivity
    folium_map.add_child(MeasureControl())
    
    return folium_map

def haversine_vectorized(lat1, lon1, lat2, lon2):
    # Radius of Earth in kilometers
    R = 6371.0
    
    # Convert to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine formula
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    # Distance
    return R * c

def add_nearby_count_vectorized(df, distance, countries):
    # Initialize the nearby count column with zeros
    title = str(distance) + "km Airports"
    df[title] = 0
    
    # Iterate through countries in the list
    for country in countries:
        # Filter the dataframe for the given country
        df_same_country = df[df['country_name'] == country]
        print(country, " - ", df_same_country.shape[0], " airports")
        
        if df_same_country.shape[0] > 1:
            # Extract latitudes and longitudes for vectorized computation
            latitudes = df_same_country['latitude_deg'].values
            longitudes = df_same_country['longitude_deg'].values
            
            # Create a grid of differences (pairwise haversine distances)
            lat1, lat2 = np.meshgrid(latitudes, latitudes)
            lon1, lon2 = np.meshgrid(longitudes, longitudes)
            distances = haversine_vectorized(lat1, lon1, lat2, lon2)
            
            # Count rows where the distance is within the radius (excluding itself)
            counts = np.sum((distances <= distance) & (distances > 0), axis=1)
            
            # Update the count in the original dataframe
            df.loc[df['country_name'] == country, title] = counts
    
    return df

# Create pd.DataFrame form ourairports.com csv download
https://ourairports.com/

In [ ]:
cwd_path = os.getcwd()
cwd_files = os.listdir()

In [ ]:
cwd_files

In [ ]:
csv_path = os.path.join(cwd_path,'world-airports.csv')
df = pd.read_csv(csv_path)

In [ ]:
df.columns

In [ ]:
df

In [ ]:
df['country_name'].unique()

# Append columns: 
1. #### Number of airports within 200 km in same country
2. #### Number of airports within 800 km in same country

In [ ]:
# Count how many countries are within 500 km of Sweden
country_list = ['Sweden','Norway','Finland','Denmark','Germany','United Kingdom','United States']
df = add_nearby_count_vectorized(df=df, distance=200, countries=country_list)
df = add_nearby_count_vectorized(df=df, distance=800, countries=country_list)

In [ ]:
df.loc[result['country_name']=='Sweden']

# Create dataframe with airport counts for given countries

In [ ]:
countries = df['country_name'].unique()

closed = []
small = []
medium = []
large = []
airports200 = []
airports800 = []

for country in countries:
    closed.append(df.loc[(df['type'] == 'closed') & (df['country_name'] == country)].shape[0])
    small.append(df.loc[(df['type'] == 'small_airport') & (df['country_name'] == country)].shape[0])
    medium.append(df.loc[(df['type'] == 'medium_airport') & (df['country_name'] == country)].shape[0])
    large.append(df.loc[(df['type'] == 'large_airport') & (df['country_name'] == country)].shape[0])
    airports200.append(df.loc[(df['country_name'] == country) & (df['200km Airports'] > 0)].shape[0])
    airports800.append(df.loc[(df['country_name'] == country) & (df['800km Airports'] > 0)].shape[0])

In [ ]:
airport_counts = pd.DataFrame({ 'Country':countries,
                                'Large Airports':large,
                                'Medium Airports':medium,
                                'Small Airports':small,
                                'Closed Airports':closed,
                                '200 km Airports':airports200,
                                '800 km Airports':airports800
                              })

In [ ]:
airport_counts.loc[airport_counts['Closed Airports']>0]

In [ ]:
airport_counts.loc[(airport_counts['Country']=='Sweden') |
                   (airport_counts['Country']=='Norway') |
                   (airport_counts['Country']=='Finland') |
                   (airport_counts['Country']=='Denmark') |
                   (airport_counts['Country']=='Germany') |
                   (airport_counts['Country']=='United Kingdom') |
                   (airport_counts['Country']=='United States')
                  ]

# Closed airports in Sweden with Folium

In [ ]:
folium_map = plot_map_with_hollow_radii(df, 'Sweden', 'closed')

In [ ]:
folium_map

# Closed airports in the United States with Folium

In [ ]:
folium_map_US = plot_map_with_hollow_radii(df, 'Sweden', 'small_airport')

In [ ]:
folium_map_US